# Programming Assignment 1: Convolution and Back-Propagation

**UBC CPEN 455: Deep Learning, 2024 Winter Term 2**

**Created By Renjie Liao**

**Date: Feb. 19, 2025**

**Name: Weifeng Ke**

**Student ID: 18879288**


This helps me to mount my google drive onto the colab. I have used chatGPT to help me find the function definitions, like how to create a tensor with zeros,how to do matrix multiplication, and how to do reshape the tensor. Also i have asked chat how to do gradient for the batch normalization, i have ask chat for the theory behind grad batch normalization, how to do back propergation for the CNN.

In [115]:
# from google.colab import drive
# drive.mount('/content/drive')
# #drive.mount('/content/drive/MyDrive/CPEN_455')

---
# Setup

We will use PyTorch to implement this assignment.

In [116]:
# Imports
import pdb
import torch
import numpy as np
from torch import nn
from math import pi
from math import floor
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets, transforms, utils

torch.set_default_dtype(torch.float64)

## load MNIST images
B = 5 # batch size
train_set = datasets.MNIST('./',
                            train=True,
                            download=True,
                            transform=transforms.ToTensor())
# train_set = datasets.MNIST('/content/drive/MyDrive/CPEN_455/data',
#                             train=True,
#                             download=False,
#                             transform=transforms.ToTensor())
loader = torch.utils.data.DataLoader(train_set, batch_size=B)

## load a batch of MNIST images as a PyTorch tensor (shape: B x C x H x W)
# B: batch size
# C: number of channels
# H: height of images
# W: width of images
img, label = next(iter(loader)) # img shape: B x C x H x W, label shape: B X 1

## create a random filter (shape: D x C x K x K)
K = 3 # kernel size
P = 1 # padding size
C = 1 # channel size
D = 2 # number of filters
filter = torch.randn(D, C, K, K) # filter shape: D x C x K x K

---
# Q1 [60Pts]: 2D convolution and its gradient




## 1.1 [30Pts]  Implement 2D convolution:

Discrete convolution can be implemented in multiple ways, e.g., matrix multiplication in spatial/Fourier domains.

First, let us take a look at 1D convolution in spatial domain. Suppose we have a 1D signal with $n$ elements $x_1, x_2, \dots, x_n$ and a 1D filter with $m$ weights $h_1, h_2, \dots, h_m$. Note that we typically use filters with odd sizes for the ease of indexing.

The (discrete) convolution with zero-padding and stride 1 is defined as:

\begin{align}
    y = h \ast x = \sum_{i=1}^{n} \sum_{j=1}^{m} h_j x_{i - \lfloor m/2 \rfloor - 1 + j},
\end{align}
where padded values $x_{-\lfloor m/2 \rfloor + 1}, \dots, x_{0}, x_{n+1}, \dots, x_{n - \lfloor m/2 \rfloor - 1 + m}$ are all zeros.

If you forget about the concepts of padding and stride, take a look at [this guide](https://arxiv.org/pdf/1603.07285.pdf) or [these pictures](https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md).

In the 1D case, we can illustrate the two matrix multiplication views of spatial convolution as below.

1.   **im2col**:
The key idea is to first extract the spatial windows from the signal $x$ for individual convolutions and then perform convolutions (i.e., dot product with the filter).
If we put each window as a column in a matrix (the right one in RHS below), then we can perform convolution via the following matrix multiplication (N.B.: the products between the filter and individual columns can be done in parallel).

\begin{align}
    y^\top = (h \ast x)^{\top} = \begin{bmatrix}
                h_m & h_{m-1} & \cdots & h_3 & h_2 & h_1
            \end{bmatrix}
            \begin{bmatrix}
                x_{m - \lfloor m/2 \rfloor} & x_{m - \lfloor m/2 \rfloor + 1} & \cdots & x_m & x_{m+1} & \cdots & 0 & 0 \\
                \vdots & \vdots & \cdots & x_{m-1} & x_m & \cdots & \vdots & \vdots \\
                x_1 & x_2 & \cdots & \vdots & x_{m-1} & \cdots  & x_n & 0 \\
                0 & x_1 & \cdots & \vdots & \vdots & \cdots  & x_{n-1} & x_n \\
                \vdots & 0 & \cdots & \vdots & \vdots & \cdots & \vdots & \vdots \\                        
                0 & 0 & \cdots & x_1 & x_2 & \cdots & x_{n - \lfloor m/2 \rfloor+1} & x_{n - \lfloor m/2 \rfloor}
            \end{bmatrix}.
\end{align}

2.   **filter2row**: The key idea is to convert the filter and the signal to a sparse cyclic matrix and a vector respectively.
Then the convolution is simply the matrix multiplication between the filter and the signal.

\begin{align}
        y = h \ast x =
            \begin{bmatrix}
                h_{\lfloor m/2 \rfloor + 1} & h_{\lfloor m/2 \rfloor + 2} & \cdots & h_m & 0 & \cdots & \cdots & \cdots & \cdots & 0 \\
                h_{\lfloor m/2 \rfloor} & h_{\lfloor m/2 \rfloor + 1} & \cdots & h_{m-1} & h_m & 0 & \cdots & \cdots & \cdots & 0 \\
                \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
                h_1 & h_2 & \cdots & \cdots & \cdots & \cdots & h_m & 0 & \cdots & 0 \\
                0 & h_1 & h_2 & \cdots & \cdots & \cdots & \cdots & h_m & \cdots & 0 \\
                \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
                0 & 0 & \cdots & \cdots & \cdots & 0 & h_1 & h_2 & \cdots & h_{\lfloor m/2 \rfloor + 2} \\                
                0 & 0 & \cdots & \cdots & & \cdots \cdots & 0 & h_1 & \cdots & h_{\lfloor m/2 \rfloor + 1}
            \end{bmatrix}
            \begin{bmatrix}
                x_1 \\
                x_2 \\
                x_3 \\
                \vdots \\
                x_n
            \end{bmatrix}
\end{align}

**Task**:
Implement the 2D convolution in the spatial domain via matrix multiplication following the above two views: **im2col** and **filter2row**.
The starter code is provided below.
You just need to fill in the missing parts of function ***conv2d_im2col*** and ***conv2d_filter2row***.
If your implementation is correct, the ***unit_test*** will output:

*Your implementation of xxx is correct!*

Otherwise, it will output:

*Your implementation of xxx is wrong!*

**N.B.**: we assume the strides along height and width are the same and the kernel is square

**Hint**: you can reduce the 2D case to 1D and follow the above construction.

In [117]:
# B,C,H,W=img.shape
# D,K=filter.shape[0],filter.shape[2]
# # img_show=img[0,0,:,:]
# # img_see=img_show.squeeze(0)
# # plt.imshow(img_see)
# # plt.show()
# # img[0]
# # print(img[0,0,:,:])
# img_pad=torch.nn.functional.pad(img, (P, P, P, P), mode='constant', value=0)
# #img_pad[0,0,:,:]
# print("image size:",img.shape)
# print("Padded image size :" ,img_pad.shape)
# print("filter size:", filter.shape)
# filter_flat=filter.reshape(D,C,K*K)
# img_pad_flat=img_pad.reshape(B,C,img_pad.shape[2]*img_pad.shape[3])

# print("filter_flat size:", filter_flat.shape)
# print("img_pad_flat size:", img_pad_flat.shape)

In [118]:
# def pre_process(img, filter, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
#     B=img.shape[0]
#     C=img.shape[1]
#     D=filter.shape[0]
#     # Pad the image using tensor
#     img_padded = torch.nn.functional.pad(img, (padding, padding, padding, padding), mode='constant', value=0) # Use torch.nn.functional.pad for padding
#     #img_padded = np.pad(img, pad_width=((0,0), (0,0), (padding, padding), (padding, padding)), mode='constant')
#     H_pad, W_pad = img_padded.shape[2], img_padded.shape[3]

#     # Calculate the dimensions of the output feature map
#     out_h = floor((H_pad - kernel_size) / stride) + 1
#     out_w = floor((W_pad - kernel_size) / stride) + 1

#     # Initialize an empty array to hold columns (each column is one patch)
#     #column size is (B X C X out_h*out_w X C*K*K) ->(B X C X 28*28 X 3*3)
#     #cols = np.zeros((B, C, out_h*out_w, kernel_size*kernel_size))

#     #Construct the column matrix
#     cols= torch.zeros(B, C , out_h*out_w, K*K)
#     for b in range(B):
#       col_idx=0 # initialize the conlumn index
#       #extract the patches
#       for i in range(0, H_pad - kernel_size + 1, stride):
#           for j in range(0, W_pad - kernel_size + 1, stride):
#               # Extract the patch (kernel_size x kernel_size)
#               patch = img_padded[b, 0, i:i + kernel_size, j:j + kernel_size]
#               # Reshape the patch and assign to the column matrix
#               cols[b,0,col_idx] = patch.flatten()
#               col_idx += 1  # Increment column index

#     cols=cols.reshape(B,C,out_h*out_w,K*K).permute(0,1,3,2)
#     print("Columns shape is:",cols.shape)
#     cols=cols.reshape(B,C*K*K,out_h*out_w)
#     print("New Columns shape is:",cols.shape)

#     filter_flat=filter.reshape(D,C*K*K)
#     print("filter_flat shape is:",filter_flat.shape)
#     #filter dimension (D X C*K*K) times img shape(B X C*K*K X out_h*out_w)
#     out=torch.matmul(filter_flat, cols)
#     out=out.reshape(B,D,out_h*out_w)
#     print("out shape is:",out.shape)

#     #print("filter flat : ", filter_flat)
#     #print("filter flat flip : ", filter_flat_flip)

# pre_process(img, filter, channel_size=C, num_filters=1, kernel_size=K, stride=1, padding=P)
# # display("the flat img has size:",flat_img.shape)
# # display("The flat filter has size :",flat_fil.shape)
# # flat_img=torch.tensor(flat_img)
# # flat_fil=torch.tensor(flat_fil)
# # display("This is the flatted filter: ",flat_fil)
# # display("This is the flatted img:", flat_img)

In [119]:
## implement the following two functions
def conv2d_im2col(img, filter, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    ### Fill in this function ###
    # Args:
    #   img: images, shape B x C x H x W
    #   filter: filters, shape D x C x K x K
    #   channel_size: number of channels, scalar (C)
    #   num_filters: number of filters, scalar (D)
    #   kernel_size: kernel size, scalar (K)
    #   stride: stride size, scalar
    #   padding: padding size, scalar
    #
    # Returns:
    #   out: convoluted images, shape B x D x H x W
    B,C,H,W=img.shape
    D,C_f,K=filter.shape[0],filter.shape[1],filter.shape[2]

    # Calculate output spatial dimensions
    out_h = floor((H + 2 * padding - kernel_size) / stride) + 1
    out_w = floor((W + 2 * padding - kernel_size) / stride) + 1
     # Pad the image using a for loop
    H_pad, W_pad = H + 2 * padding, W + 2 * padding
    img_padded = torch.zeros(B, C, H_pad, W_pad)
    for b in range(B):
        for c in range(C):
            # Copy the original image to the center of the padded image
            img_padded[b, c, padding:H_pad - padding, padding:W_pad - padding] = img[b, c]

    filter_flat=filter.reshape(D,C*K*K)


    #Construct the column matrix
    cols= torch.zeros(B, out_h*out_w, C*K*K)
    for b in range(B):
      col_idx=0 # initialize the conlumn index
      #extract the patches
      for i in range(0, H_pad - kernel_size + 1, stride):
          for j in range(0, W_pad - kernel_size + 1, stride):
              # Extract the patch (c,kernel_size x kernel_size)
              patch = img_padded[b, :, i:i + kernel_size, j:j + kernel_size]
              # Reshape the patch and assign to the column matrix
              cols[b,col_idx,:] = patch.reshape(-1)
              col_idx += 1  # Increment column index
    # Rearrange cols to shape (B, C*K*K, H_out*W_out) by transposing the last two dimensions
    cols = cols.permute(0, 2, 1)  # Now shape: (B, C*K*K, H_out*W_out)
    # For each image in the batch, perform matrix multiplication:
    out = torch.zeros(B, D, out_h * out_w)
    for b in range(B):
        out[b] = torch.matmul(filter_flat, cols[b])

    # Reshape output to (B, D, H_out, W_out)
    out = out.reshape(B, D, out_h, out_w)
    return out

def conv2d_filter2row(img, filter, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    ### Fill in this function ###
    # Args:
    #   img: images, shape B x C x H x W
    #   filter: filters, shape D x C x K x K
    #   channel_size: number of channels, scalar (C)
    #   num_filters: number of filters, scalar (D)
    #   kernel_size: kernel size, scalar (K)
    #   stride: stride size, scalar
    #   padding: padding size, scalar
    #
    # Returns:
    #   out: convoluted images, shape B x D x H x W

    # Calculate output spatial dimensions
    B,C,H,W=img.shape
    D,C_f,K=filter.shape[0],filter.shape[1],filter.shape[2]

    # Calculate output spatial dimensions
    out_h = floor((H + 2 * padding - kernel_size) / stride) + 1
    out_w = floor((W + 2 * padding - kernel_size) / stride) + 1
     # Pad the image using a for loop
    H_pad, W_pad = H + 2 * padding, W + 2 * padding
    img_padded = torch.zeros(B, C, H_pad, W_pad)
    for b in range(B):
        for c in range(C):
            # Copy the original image to the center of the padded image
            img_padded[b, c, padding:H_pad - padding, padding:W_pad - padding] = img[b, c]

    # So, we'll initialize cols with shape: (B, H_out*W_out, C*K*K)
    cols = torch.zeros(B, out_h * out_w, C * kernel_size * kernel_size, device=img.device, dtype=img.dtype)

    #flaten the filter into column matrix shape (C*K*K, D)
    filter_row=filter.reshape(D,filter.shape[1]*filter.shape[2]*filter.shape[3]).T
    #print("filter_row shape is:",filter_row.shape)

    # Pad the image using a for loop
    #Construct the row matrix
    rows = torch.zeros(B, out_h*out_w, filter.shape[1]*filter.shape[2]*filter.shape[3])
    #print("Shape of the rows:", rows.shape)
    for b in range(img.shape[0]):
      row_idx=0 # initialize the row index
      #extract the patches
      for i in range(0, H + 2 * padding - kernel_size + 1, stride):
          for j in range(0, W + 2 * padding - kernel_size + 1, stride):
              # Extract the patch (kernel_size x kernel_size)
              patch = img_padded[b, :, i:i + kernel_size, j:j + kernel_size]
              # Reshape the patch and assign to the column matrix
              rows[b, row_idx,:] = patch.flatten() #reshape to size C*K*K
              row_idx += 1  # Increment row index
    #print("Shape of the filter rows:", filter_row.shape)
    #print("Shape of the rows:", rows.shape)
    # For each image in the batch, perform the matrix multiplication:
    # (out_h*out_w, C*K*K) dot (C*K*K, D) -> (out_h*out_w, D)
    out = torch.matmul(rows, filter_row)

    # Reshape to the output (B, out_h, out_w, D)
    out = out.reshape(B, out_h, out_w, D).permute(0,3,1,2)
    #print("size of out", out.shape)
    return out

def unit_test_conv2d(img, filter, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    # call your implemented "im2col" conv2D
    y_im2col = conv2d_im2col(img, filter, channel_size=channel_size, num_filters=num_filters, kernel_size=kernel_size, stride=stride, padding=padding)

    # ground truth conv2D
    y_gt = F.conv2d(img, filter, stride=stride, padding=padding)

    diff = (y_im2col - y_gt).norm()
    if diff < 1.0e-9:
        print("Your implementation of conv2d_im2col is correct!")
    else:
        print("Your implementation of conv2d_im2col is wrong!")

    # call your implemented "im2col" conv2D
    y_filter2row = conv2d_filter2row(img, filter, channel_size=channel_size, num_filters=num_filters, kernel_size=kernel_size, stride=stride, padding=padding)

    diff = (y_filter2row - y_gt).norm()
    if diff < 1.0e-9:
        print("Your implementation of conv2d_filter2row is correct!")
    else:
        print("Your implementation of conv2d_filter2row is wrong!")


unit_test_conv2d(img, filter, channel_size=C, num_filters=D, kernel_size=K, stride=1, padding=P)

Your implementation of conv2d_im2col is correct!
Your implementation of conv2d_filter2row is correct!


## 1.2 [20Pts] Implement the gradient of 2D convolution

We now turn to the gradient of 2D convolution.
In particular, given a batch of images $x$ with the shape $B \times C \times H \times W$ and filters $h$ with shape $D \times C \times K \times K$, we can view the convolution (zero-padding and stride 1) as a function
\begin{align}
    y = f(h, x),
\end{align}
that would produce an output tensor $y$ with shape $B \times D \times H \times W$.

If we vectorize $x$, $h$, and $y$, then the Jacobian matrix $∇f = [\frac{\partial y}{\partial h}, \frac{\partial y}{\partial x}]$ would be of shape $BDHW \times (DCKK + BCHW)$.
In practice, we almost never need to compute the Jacobian matrix directly as it is unnecessary for back-propagation.
Instead, we often need to compute the product between the transposed Jacobian and a vector (a.k.a. vector-Jacobian product), i.e., ${\frac{\partial y}{\partial h}}^{\top} v$ and ${\frac{\partial y}{\partial x}}^{\top} v$.
For example, the vector $v$ could the gradient of some loss $\ell$ (scalar) w.r.t. the output above (i.e., $\frac{\partial \ell}{\partial y}$).

**Task**:
Given input images $x$, filters $h$, output $y$, and a vector $v$, implement the gradients ${\frac{\partial y}{\partial h}}^{\top} v$ and ${\frac{\partial y}{\partial x}}^{\top} v$ in the function below.

**N.B.**: The function needs to return the gradients in the original shapes, i.e., ${\frac{\partial y}{\partial h}}^{\top} v$ should have the same shape as $h$ ($D \times C \times K \times K$) and ${\frac{\partial y}{\partial x}}^{\top} v$ should have the same shape as $x$ ($B \times C \times H \times W$).

In [120]:
def im2col(img, kernel_size, stride, padding):
    B, C, H, W = img.shape
    out_h = (H + 2*padding - kernel_size) // stride + 1
    out_w = (W + 2*padding - kernel_size) // stride + 1

    # Pad the image using a for loop (as before)
    H_pad, W_pad = H + 2 * padding, W + 2 * padding
    img_padded = torch.zeros(B, C, H_pad, W_pad, dtype=img.dtype, device=img.device)
    for b in range(B):
        for c in range(C):
            img_padded[b, c, padding:H_pad - padding, padding:W_pad - padding] = img[b, c]

    # Construct the column matrix using for loops
    cols = torch.zeros(B, C * kernel_size * kernel_size, out_h * out_w, dtype=img.dtype, device=img.device)
    for b in range(B):
        col_idx = 0
        for i in range(0, H_pad - kernel_size + 1, stride):
            for j in range(0, W_pad - kernel_size + 1, stride):
                # Extract the patch and flatten it
                patch = img_padded[b, :, i:i + kernel_size, j:j + kernel_size]  # Extract all channels
                cols[b, :, col_idx] = patch.flatten()
                col_idx += 1

    return cols
def col2im(cols, img_shape, kernel_size, stride, padding):
    B, C, H, W = img_shape
    # Use F.fold to invert the unfolding
    H_pad, W_pad = H + 2 * padding, W + 2 * padding
    # Initialize the output image with zeros
    img_padded = torch.zeros(B, C, H_pad, W_pad, dtype=cols.dtype, device=cols.device)

    # Iterate through the columns and place patches back into the image
    col_idx = 0
    for i in range(0, H_pad - kernel_size + 1, stride):
        for j in range(0, W_pad - kernel_size + 1, stride):
            # Extract the column and reshape it to a patch
            patch = cols[:, :, col_idx].view(B, C, kernel_size, kernel_size)
            # Add the patch to the corresponding location in the image
            img_padded[:, :, i:i + kernel_size, j:j + kernel_size] += patch
            col_idx += 1
    # Remove the extra padding
    if padding > 0:
        img = img_padded[:, :, padding:-padding, padding:-padding]
    else:
        img = img_padded
    return img

# implement the following functions
def grad_conv2d(img, filter, out, grad_out, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    ### Fill in this function ###
    # Args:
    #   img: images, shape B x C x H x W
    #   filter: filters, shape D x C x K x K
    #   out: convoluted images, shape B x D x H x W
    #   grad_out: gradient w.r.t. output, shape B x D x H x W
    #   channel_size: number of channels, scalar (C)
    #   num_filters: number of filters, scalar (D)
    #   kernel_size: kernel size, scalar (K)
    #   stride: stride size, scalar
    #   padding: padding size, scalar
    #
    # Returns:
    #   grad_img: gradient w.r.t. img, shape B x C x H x W
    #   grad_filer: gradient w.r.t. filter, shape D x C x K x K

    #getting the right shape
    B, C, H, W = img.shape
    D, C_f, K, K2 = filter.shape
    # Compute output spatial dimensions.
    out_h = (H + 2*padding - kernel_size) // stride + 1
    out_w = (W + 2*padding - kernel_size) // stride + 1


    # Compute gradient w.r.t. filters
    # Unroll input using im2col: shape (B, C*K*K, out_h*out_w)
    cols = im2col(img, kernel_size, stride, padding)
    #print("im2col output shape: ",cols.shape)
    # Reshape grad_out: (B, D, out_h*out_w)
    grad_out_reshaped = grad_out.view(B, D, -1)

    # We want grad_filter of shape (D, C*K*K). For each image:
    # grad_filter += grad_out[b] (D x (out_h*out_w)) @ cols[b].T ((out_h*out_w) x (C*K*K))
    grad_filter = torch.zeros(D, C * kernel_size * kernel_size, device=img.device, dtype=img.dtype)
    for b in range(B):
        grad_filter += torch.matmul(grad_out_reshaped[b], cols[b].transpose(0,1))
    # Reshape to (D, C, K, K)
    grad_filter = grad_filter.view(D, C, kernel_size, kernel_size)

    # --- Compute gradient w.r.t. input images ---
    # Reshape filters to (D, C*K*K)
    filt_reshaped = filter.view(D, -1)
    # For each image, compute the column form gradient:
    # grad_img_cols = (filt_reshapedᵀ) (shape: (C*K*K, D)) @ grad_out[b] (D x (out_h*out_w))
    grad_img_cols = torch.matmul(filt_reshaped.transpose(0,1), grad_out_reshaped)
    # grad_img_cols has shape (B, C*K*K, out_h*out_w)
    # Fold the columns back into the image using col2im
    grad_img = col2im(grad_img_cols, img.shape, kernel_size, stride, padding)

    return grad_img, grad_filter

def unit_test_grad_conv2d(img, filter, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    filter.requires_grad = True
    img.requires_grad = True

    ### ground truth conv2D
    img_out = F.conv2d(img, filter, stride=stride, padding=padding)

    # create a random vector v
    v = torch.randn_like(img_out)

    # call your implemented "grad_conv2d" function
    grad_img, grad_filter = grad_conv2d(img, filter, img_out, v, channel_size=channel_size, num_filters=num_filters, kernel_size=kernel_size, stride=stride, padding=padding)

    # compute ground-truth gradients
    grad_img_gt = torch.autograd.grad(img_out, img, grad_outputs=v, retain_graph=True)[0]
    grad_filter_gt = torch.autograd.grad(img_out, filter, grad_outputs=v, retain_graph=True)[0]
    #pdb.set_trace()

    diff = (grad_img - grad_img_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of grad_img is correct!")
    else:
        print("Your implementation of grad_img is wrong!")

    diff = (grad_filter - grad_filter_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of grad_filter is correct!")
    else:
        print("Your implementation of grad_filter is wrong!")

unit_test_grad_conv2d(img, filter, channel_size=C, num_filters=D, kernel_size=K, stride=1, padding=P)

Your implementation of grad_img is correct!
Your implementation of grad_filter is correct!


---
## 1.3 [10Pts]: Implement gradient checking via the finite difference approximation

We verify the correctness of the implementation of gradient operators by calling PyTorch's autograd function.
However, PyTorch's autograd function just calls the gradient operators implemented by the PyTorch team.
How do they verify the correctness of their implementation?

The answer is **finite difference approximation**.
Following the setup in 1.2, given a batch of images $x$ with the shape $B \times C \times H \times W$ and filters $h$ with shape $D \times C \times K \times K$, we have the convolution (zero-padding and stride 1)
\begin{align}
    y = f(h, x).
\end{align}

Again, mentally vectorize $x$ and $y$ would help us understand the math.
Given any vector $v$ with the same shape as $y$, we are interested in computing ${\frac{\partial y}{\partial h}}^{\top} v$ and ${\frac{\partial y}{\partial x}}^{\top} v$.
These two gradients are equivalent to ${\frac{\partial \ell}{\partial h}}$ and ${\frac{\partial \ell}{\partial x}}$ where
\begin{align}
    \ell(h, x) = y^{\top}v = f(h, x)^{\top} v.
\end{align}
Note that here $\ell$ becomes a scalar.
Based on Talyor's theorem, we have
\begin{align}
    d^{\top} \frac{\partial \ell}{\partial h} = \lim_{\epsilon → 0} \frac{\ell(h + \epsilon \cdot d, x) - \ell(h - \epsilon \cdot d, x)}{2 ϵ},
\end{align}
where $d$ could be any direction vector and $ϵ$ is a scalar.
For our purpose, we just need to set $d$ to be the unit vector to compute the per-dimension value of $\frac{\partial \ell}{\partial h}$.
Specifically, if we set $d$ as the $i$-th unit vector $e_i$, i.e., $d[i] = 1$ and $d[j] = 0, \forall j \neq i$, we can then compute
\begin{align}
    \frac{\partial \ell}{\partial h}[i] &= \lim_{\epsilon → 0} \frac{\ell(h + \epsilon \cdot e_i, x) - \ell(h - \epsilon \cdot e_i, x)}{2 ϵ} \\
    & ≈ \frac{\ell(h + \epsilon \cdot e_i, x) - \ell(h - \epsilon \cdot e_i, x)}{2 ϵ}.
\end{align}

**Task**: Implement the finite-difference based gradient checker for ${\frac{\partial \ell}{\partial h}}$ and ${\frac{\partial \ell}{\partial x}}$.


**N.B.**: For efficiency consideration in the unit test, you can use F.conv2d to compute the convolution in your implementation of *grad_checker*. This assignment is to let you understand how to implement finte-difference. But in pratice, if we want to verify our implementation of conv2d, then we should use our conv2d instead of F.conv2d from PyTorch.


In [121]:
## implement the following functions
def loss_fn(img, filter, conv, grad_out):
    y = conv(img, filter,padding=1)
    loss = torch.sum(y * grad_out)
    return loss
def grad_checker(img, filter, conv, grad_out, epsilon=1.0e-5, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    ### Fill in this function ###
    # Args:
    #   img: images, shape B x C x H x W
    #   filter: filters, shape D x C x K x K
    #   conv: convolution function
    #   out: convoluted images, shape B x D x H x W
    #   grad_out: gradient w.r.t. output, shape B x D x H x W
    #   channel_size: number of channels, scalar (C)
    #   num_filters: number of filters, scalar (D)
    #   kernel_size: kernel size, scalar (K)
    #   stride: stride size, scalar
    #   padding: padding size, scalar
    #
    # Returns:
    #   grad_img: gradient w.r.t. img, shape B x C x H x W
    #   grad_filer: gradient w.r.t. filter, shape D x C x K x K

    img_og=img.clone()
    filter_og=filter.clone()

    # Initialize numerical gradients with zeros.
    grad_filter_fd = torch.zeros_like(filter)
    grad_img_fd    = torch.zeros_like(img)
  	# Gradient with respect to filter, Flatten filter for easier indexing.
    filt_flat = filter.view(-1)
    for i in range(filt_flat.numel()):
        # Create a perturbation vector for filt.
        perturb = torch.zeros_like(filt_flat)
        perturb[i] = epsilon

        # Evaluate loss for filt + epsilon*e_i
        l_plus = loss_fn(img_og, (filt_flat + perturb).view_as(filter), conv, grad_out)
        # Evaluate loss for filt - epsilon*e_i
        l_minus = loss_fn(img_og, (filt_flat - perturb).view_as(filter), conv, grad_out)

        # Finite difference approximation.
        grad_filter_fd.view(-1)[i] = (l_plus - l_minus) / (2 * epsilon)
    # --- Gradient with respect to input image ---
    # Flatten img for easier indexing.
    img_flat = img.view(-1)
    for i in range(img_flat.numel()):
        # Create a perturbation vector for img.
        perturb = torch.zeros_like(img_flat)
        perturb[i] = epsilon

        # Evaluate loss for img + epsilon*e_i
        l_plus = loss_fn((img_flat + perturb).view_as(img), filter_og, conv, grad_out)
        # Evaluate loss for img - epsilon*e_i
        l_minus = loss_fn((img_flat - perturb).view_as(img), filter_og, conv, grad_out)

        # Finite difference approximation.
        grad_img_fd.view(-1)[i] = (l_plus - l_minus) / (2 * epsilon)

    # Return the numerical gradients (convert back to float if desired)
    return grad_img_fd.float(), grad_filter_fd.float()

def unit_test_grad_checker(img, filter, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    epsilon = 1.0e-5
    filter.requires_grad = True
    img.requires_grad = True

    ### ground truth conv2D
    img_out = F.conv2d(img, filter, stride=stride, padding=padding)

    # create a random vector v
    v = torch.randn_like(img_out)

    # call your implemented "grad_checker" function
    grad_img, grad_filter = grad_checker(img, filter, F.conv2d, v, epsilon=epsilon, channel_size=channel_size, num_filters=num_filters, kernel_size=kernel_size, stride=stride, padding=padding)

    # compute ground-truth gradients
    grad_img_gt = torch.autograd.grad(img_out, img, grad_outputs=v, retain_graph=True)[0]
    grad_filter_gt = torch.autograd.grad(img_out, filter, grad_outputs=v, retain_graph=True)[0]
    #pdb.set_trace()

    diff = (grad_img - grad_img_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of grad_img is correct!")
    else:
        print("Your implementation of grad_img is wrong!")

    diff = (grad_filter - grad_filter_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of grad_filter is correct!")
    else:
        print("Your implementation of grad_filter is wrong!")

unit_test_grad_checker(img, filter, channel_size=C, num_filters=D, kernel_size=K, stride=1, padding=P)

Your implementation of grad_img is correct!
Your implementation of grad_filter is correct!


---
#Q2 [5Pts]: Implement ReLU and its gradient

**Task**: Implement ReLU operator, i.e., $f(x) = max(x, 0)$, and its gradient operator ${\frac{\partial f}{\partial x}}^{\top} v$ for any given tensor $v$ that is of the same shape as $x$.

**N.B.**: For simplicity, we can assume the input $x$ is of shape $B \times C \times H \times W$ as before.

In [122]:
## implement the following functions
def func_relu(x):
    ### Fill in this function ###
    # Args:
    #   x: input, shape B x C x H x W
    #
    # Returns:
    #   y: output, shape B x C x H x W
    B,C,H,W=x.shape
    y=torch.zeros(B,C,H,W)
    for i in range(B):
      for j in range(C):
        for k in range(H):
          for l in range(W):
            if x[i,j,k,l]>0:
              y[i,j,k,l]=x[i,j,k,l]
            else:
              y[i,j,k,l]=0
    return y

def grad_relu(x, y, grad_out):
    ### Fill in this function ###
    # Args:
    #   x: input, shape B x C x H x W
    #   y: output, shape B x C x H x W
    #   grad_out: gradient w.r.t. output y, shape B x D x H x W
    #
    # Returns:
    #   grad_x: gradient w.r.t. x, shape B x C x H x W

    #compute the gradient for y=relu(X) if x> 0 gradient of x = 1* grad_out
    #del loss/del y * del y/del x* del Relu(x)/x
    B,C,H,W=x.shape
    grad_x=torch.zeros(B,C,H,W)
    for i in range(B):
      for j in range(C):
        for k in range(H):
          for l in range(W):
            if x[i,j,k,l]>0.0:
              grad_x[i,j,k,l]=1*grad_out[i,j,k,l]
            else:
              grad_x[i,j,k,l]=0.0
    return grad_x
# def grad_relu(x, y, grad_out):
#     return grad_out * (x > 0).float()
def unit_test_relu(x):
    x.requires_grad = True

    # call your implemented "func_relu" function
    y = func_relu(x)

    # ground truth ReLU
    y_gt = F.relu(x)

    diff = (y - y_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of func_relu is correct!")
    else:
        print("Your implementation of func_relu is wrong!")

    # create a random vector v
    v = torch.randn_like(y)

    # call your implemented "grad_relu" function
    grad_x = grad_relu(x, y, v)

    # compute ground-truth gradients
    grad_x_gt = torch.autograd.grad(y_gt, x, grad_outputs=v, retain_graph=True)[0]

    diff = (grad_x - grad_x_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of grad_relu is correct!")
    else:
        print("Your implementation of grad_relu is wrong!")

unit_test_relu(torch.randn_like(img))

Your implementation of func_relu is correct!
Your implementation of grad_relu is correct!


---
#Q3 [20Pts]: Implement Batch-normalization (BN) for convolution and its gradient

Given a batch of input images $x$ with shape $B \times C \times H \times W$, we compute a single mean and a single standard deviation per channel as below,
\begin{align}
    \mu[c] &= \frac{1}{BHW} \sum_{i=1}^{B} \sum_{m=1}^{H} \sum_{n=1}^{W} x[i, c, m, n] \\
    \sigma^2[c] &= \frac{1}{BHW} \sum_{i=1}^{B} \sum_{m=1}^{H} \sum_{n=1}^{W} (x[i, c, m, n] - \mu[c])^2.
\end{align}
Then we perform BN, $y = f(x, \beta, \gamma)$, as,
\begin{align}
    y[i,c,m,n] &= \gamma[c] \frac{x[i,c,m,n] - \mu[c]}{\sqrt{\sigma^2[c] + \epsilon}} + \beta[c],
\end{align}
where $\gamma$ and $\beta$ are learnable parameters are of shape $C$.
$ϵ$ is a constant.

**Task**: For simplicity, we fix the learnable parameters as $\gamma = 1$ and $\beta = 0$.
Implement BN for convolution and its gradient operators ${\frac{\partial f}{\partial x}}^{\top} v$ for any $v$ that is compatible with the matrix multiplication.



In [123]:
## implement the following functions
def func_batch_norm(x, epsilon=1.0e-5):
    ### Fill in this function ###
    # Args:
    #   x: input, shape B x C x H x W
    #   epsilon: constant, scalar
    #
    # Returns:
    #   y: output, shape B x C x H x W
    B,C,H,W=x.shape
    u=torch.zeros(C)
    sigma2=torch.zeros(C)
    y=torch.zeros(B,C,H,W)
    gamma=1
    beta=0
    avg=1/(B*H*W)
    #compute the means
    for c in range(C):
      for i in range(B):
        for m in range(H):
          for n in range(W):
            u[c]+=x[i,c,m,n]
      u[c]=avg*u[c]

      # Calculate sigma2[c]
      for i in range(B):
          for m in range(H):
              for n in range(W):
                  sigma2[c] += (x[i, c, m, n] - u[c])**2
      sigma2[c] = avg * sigma2[c]

      # Perform Batch Normalization
      for i in range(B):
          for m in range(H):
              for n in range(W):
                  y[i, c, m, n] = (gamma * (x[i, c, m, n] - u[c]) / torch.sqrt(sigma2[c] + epsilon))+beta
    return y

def grad_batch_norm(x, y, grad_out, epsilon=1.0e-5):
    ### Fill in this function ###
    # Args:
    #   x: input, shape B x C x H x W
    #   y: output, shape B x C x H x W
    #   grad_out: gradient w.r.t. output y, shape B x D x H x W
    #   epsilon: constant, scalar
    #
    # Returns:
    #   grad_x: gradient w.r.t. x, shape B x C x H x W
    #   grad_x=del loss / del x
    # Step 1: Gather the dimensions of the input

    # 1. Get dimensions and compute N = number of elements per channel.
    B, C, H, W = x.shape
    N = B * H * W  # Number of elements per channel
    mean=torch.zeros(C)
    var=torch.zeros(C)
    grad_x=torch.zeros_like(x)
    # 2. Compute the mean μ and variance σ² for each channel (across batch, height, width).
    for c in range(C):
        s = 0.0
        for b in range(B):
            for i in range(H):
                for j in range(W):
                    s += x[b, c, i, j]
        mean[c] = s / N

        # Now compute variance for channel c.
        s_sq = 0.0
        for b in range(B):
            for i in range(H):
                for j in range(W):
                    diff = x[b, c, i, j] - mean[c]
                    s_sq += diff * diff
        var[c] = s_sq / N

    # 3. Compute the standard deviation with epsilon
    SD = torch.sqrt(var + epsilon)
    S1 = torch.zeros(C)
    S2 = torch.zeros(C)

    for c in range(C):
        for b in range(B):
            for i in range(H):
                for j in range(W):
                    # delta is the lost wrt y gradient at this element.
                    delta = grad_out[b, c, i, j]
                    S1[c] += delta
                    # x_hat = (x - mean) / std for channel c. formalize imput x_hat
                    x_hat = (x[b, c, i, j] - mean[c]) / SD[c]

                    S2[c] += delta * x_hat


    for c in range(C):
        for b in range(B):
            for i in range(H):
                for j in range(W):
                    delta = grad_out[b, c, i, j]
                    x_hat = (x[b, c, i, j] - mean[c]) / SD[c]
                    grad_val = (delta - S1[c]/N - x_hat * (S2[c]/N)) / SD[c]
                    grad_x[b, c, i, j] = grad_val

    return grad_x

def unit_test_batch_norm(x):
    x.requires_grad = True
    epsilon = 1e-5

    # call your implemented "func_batch_norm" function
    y = func_batch_norm(x, epsilon=epsilon)

    # ground truth ReLU
    BN_gt = nn.BatchNorm2d(x.shape[1], eps=epsilon, momentum=1.0, affine=False, track_running_stats=False)
    y_gt = BN_gt(x)

    diff = (y - y_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of func_batch_norm is correct!")
    else:
        print("Your implementation of func_batch_norm is wrong!")

    # create a random vector v
    v = torch.randn_like(y)

    # call your implemented "grad_batch_norm" function
    grad_x = grad_batch_norm(x, y, v, epsilon=epsilon)

    # compute ground-truth gradients
    grad_x_gt = torch.autograd.grad(y_gt, x, grad_outputs=v, retain_graph=True)[0]

    diff = (grad_x - grad_x_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of grad_batch_norm is correct!")
    else:
        print("Your implementation of grad_batch_norm is wrong!")

unit_test_batch_norm(torch.randn_like(img))

Your implementation of func_batch_norm is correct!
Your implementation of grad_batch_norm is correct!


---
#Q4 [15Pts]: Implement a simple CNN and back-propagation (BP)

Now we are ready to build a deep CNN and learn it with back-propagation.
In particular, let us build a simple CNN with following architecture:

Conv $→$ BN $→$ ReLU $→$ Conv $→$ BN $→$ ReLU $→$ Linear.

Here, for all layers, the convolutions are the same as before (i.e., kernel size $3 \times 3$, zero-padding, number of filters $D = 2$, and stride 1), the BNs are without learnable $\gamma$ and $\beta$, and the last linear layer would map whatever input dimension to $10$ classes in MNIST.

**Task**: Implement the above CNN, compute the cross-entropy loss, and compute gradient of the loss w.r.t. filter weights.

**N.B.**: You can use F.cross_entropy provided by PyTorch. But for other operators like Conv, BN, and ReLU and their gradietns, you should use your previous implementations.

In [124]:
## implement the following two functions
def CNN(img, filter_1, filter_2, weight, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    ### Fill in this function ###
    # Args:
    #   img: images, shape B x C x H x W
    #   filter_1: filters at 1st layer, shape D x C x K x K
    #   filter_2: filters at 2nd layer, shape D x C x K x K
    #   weight: weights of linear readout layer, shape ? x 10
    #   channel_size: number of channels, scalar (C)
    #   num_filters: number of filters, scalar (D)
    #   kernel_size: kernel size, scalar (K)
    #   stride: stride size, scalar
    #   padding: padding size, scalar
    #
    # Returns:
    #   out: logits, shape B x 10

    B,C,H,W=img.shape
    D_1,C_1,K,K=filter_1.shape
    D_2,C_2,_,_=filter_2.shape
    classes=weight.shape[1]
    out=torch.zeros(B,classes)

    ### 1st layer
    layer_1=conv2d_im2col(img,filter_1,channel_size=C_1,num_filters=D_1,kernel_size=3,stride=1,padding=1)
    layer_1=func_batch_norm(layer_1)
    layer_1=func_relu(layer_1)
    # The output of the previous layer is layer_1 which after flattening has shape (B, D_1 * out_h * out_w).
    # Given B=5, D=2, K=3, P=1, S=1, H=28, W=28,
    # out_h = floor((H + 2*P - K) / S) + 1 = floor((28 + 2*1 - 3) / 1) + 1 = 28
    # out_w = floor((W + 2*P - K) / S) + 1 = floor((28 + 2*1 - 3) / 1) + 1 = 28
    #The layer 1 output shape is (B,D_1,28*28)=(5,2,28,28)
    #print("layer 1 shape",layer_1.shape)

    ### 2nd layer
    layer_2=conv2d_im2col(layer_1,filter_2,channel_size=C_2,num_filters=D_2,kernel_size=3,stride=1,padding=1)
    # The output of the previous layer is layer_2 which after flattening has shape (B, D * out_h * out_w).
    # Given B=5, D=2, K=3, P=1, S=1, H=28, W=28,
    # out_h = floor((H + 2*P - K) / S) + 1 = floor((28 + 2*1 - 3) / 1) + 1 = 28
    # out_w = floor((W + 2*P - K) / S) + 1 = floor((28 + 2*1 - 3) / 1) + 1 = 28
    #the output of layer_2 has shape of (5,2,28,28)
    layer_2=func_batch_norm(layer_2)
    layer_2=func_relu(layer_2)
    #print("layer 2 shape",layer_2.shape)

    ### linear readout
    #flatten the input
    #after flattening the input shape of (5,2,28,28)= (5,2*28*28)=(5,1568)
    linear_layer=layer_2.reshape(B,D_2*H*W)
    #print("linear layer shape",linear_layer.shape)
    #so to make the multiplication work (5 x 1568) @ (1568 , 10)
    out=torch.matmul(linear_layer,weight)
    #print("out shape",out.shape)


    return out

def grad_CNN(img, filter_1, filter_2, weight, grad_loss, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    B, C, H, W = img.shape

    ### 1st layer forward
    layer_1 = conv2d_im2col(img, filter_1, channel_size=filter_1.shape[1], num_filters=filter_1.shape[0],
                            kernel_size=kernel_size, stride=stride, padding=padding)
    #print("layer 1 shape",layer_1.shape)
    bn_layer1 = func_batch_norm(layer_1)
    #print("bn layer 1 shape",bn_layer1.shape)
    relu_layer1 = func_relu(bn_layer1)
    #print("relu layer 1 shape",relu_layer1.shape)

    ### 2nd layer forward
    layer_2 = conv2d_im2col(relu_layer1, filter_2, channel_size=filter_2.shape[1], num_filters=filter_2.shape[0],
                            kernel_size=kernel_size, stride=stride, padding=padding)
    #print("layer 2 shape",layer_2.shape)
    bn_layer2 = func_batch_norm(layer_2)
    #print("bn layer 2 shape",bn_layer2.shape)
    relu_layer2 = func_relu(bn_layer2)
    #print("relu layer 2 shape",relu_layer2.shape)

    ### linear readout forward
    linear_layer = relu_layer2.reshape(B, filter_2.shape[0]*img.shape[2]*img.shape[3])
    #print("linear layer shape",linear_layer.shape)
    out = torch.matmul(linear_layer, weight)
    #print("out shape",out.shape)

    ### Backward pass----------------------------------------
    # Gradient w.r.t. weight
    grad_weight = torch.matmul(linear_layer.T, grad_loss)
    #print("grad weight shape",grad_weight.shape)

    # Gradient w.r.t. linear layer input
    grad_linear = grad_loss.matmul(weight.T)
    #print("grad linear shape",grad_linear.shape)

    # Backprop through linear layer
    D2 = filter_2.shape[0]
    grad_relu_layer2 = grad_linear.reshape(B, D2, H, W)
    #print("grad relu layer 2 shape",grad_relu_layer2.shape)

    # Backprop through second ReLU
    grad_bn_layer2 = grad_relu(bn_layer2, relu_layer2, grad_relu_layer2)
    #print("grad bn layer 2 shape",grad_bn_layer2.shape)

    # Backprop through second BN (corrected argument order)
    grad_layer2 = grad_batch_norm(layer_2, bn_layer2, grad_bn_layer2)
    #print("grad layer 2 shape",grad_layer2.shape)

    # Backprop through second conv
    grad_relu_layer1, grad_filter_2 = grad_conv2d(relu_layer1, filter_2, layer_2, grad_layer2,
                                                  channel_size=filter_2.shape[1], num_filters=filter_2.shape[0],
                                                  kernel_size=kernel_size, stride=stride, padding=padding)
    #print("grad relu layer 1 shape",grad_relu_layer1.shape)
    #print("grad filter 2 shape",grad_filter_2.shape)

    # Backprop through first ReLU
    grad_bn_layer1 = grad_relu(bn_layer1, relu_layer1, grad_relu_layer1)
    #print("grad bn layer 1 shape",grad_bn_layer1.shape)

    # Backprop through first BN (corrected argument order)
    grad_layer1 = grad_batch_norm(layer_1, bn_layer1, grad_bn_layer1)
    #print("grad layer 1 shape",grad_layer1.shape)

    # Backprop through first conv
    grad_img, grad_filter_1 = grad_conv2d(img, filter_1, layer_1, grad_layer1,
                                          channel_size=filter_1.shape[1], num_filters=filter_1.shape[0],
                                          kernel_size=kernel_size, stride=stride, padding=padding)
    #print("grad img shape",grad_img.shape)
    #print("grad filter 1 shape",grad_filter_1.shape)

    # Return gradients
    return grad_filter_1, grad_filter_2, grad_weight

def unit_test_CNN(img, label, filter_1, filter_2, weight, channel_size=1, num_filters=1, kernel_size=3, stride=1, padding=1):
    # call your implemented "CNN"
    img.requires_grad_()
    filter_1.requires_grad_()
    filter_2.requires_grad_()
    weight.requires_grad_()
    y = CNN(img, filter_1, filter_2, weight, channel_size=channel_size, num_filters=num_filters, kernel_size=kernel_size, stride=stride, padding=padding)
    y.requires_grad_()

    # compute loss function
    loss = F.cross_entropy(y, label).mean()
    loss.requires_grad_()

    # compute gradient of loss w.r.t. logits
    grad_loss = torch.autograd.grad(loss, y, retain_graph=True)[0]

    # call your implemented "grad_batch_norm" function
    grad_filter_1, grad_filter_2, grad_weight = grad_CNN(img, filter_1, filter_2, weight, grad_loss, channel_size=channel_size, num_filters=num_filters, kernel_size=kernel_size, stride=stride, padding=padding)

    # compute ground-truth gradients
    grad_filter_1_gt = torch.autograd.grad(loss, filter_1, retain_graph=True)[0]
    grad_filter_2_gt = torch.autograd.grad(loss, filter_2, retain_graph=True)[0]
    grad_weight_gt = torch.autograd.grad(loss, weight, retain_graph=True)[0]

    diff = (grad_filter_1 - grad_filter_1_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of grad_filter_1 is correct!")
    else:
        print("Your implementation of grad_filter_1 is wrong!")

    diff = (grad_filter_2 - grad_filter_2_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of grad_filter_2 is correct!")
    else:
        print("Your implementation of grad_filter_2 is wrong!")

    diff = (grad_weight - grad_weight_gt).norm()
    if diff < 1.0e-5:
        print("Your implementation of grad_weight is correct!")
    else:
        print("Your implementation of grad_weight is wrong!")


filter_1 = torch.randn(D, C, K, K) # filter shape: D x C x K x K
filter_2 = torch.randn(D, D, K, K) # filter shape: D x C x K x K

### compute the correct shape and then replace None with it ###
weight = torch.randn(1568, 10) # weight of the last linear layer

unit_test_CNN(img, label, filter_1, filter_2, weight, channel_size=C, num_filters=D, kernel_size=K, stride=1, padding=P)

Your implementation of grad_filter_1 is correct!
Your implementation of grad_filter_2 is correct!
Your implementation of grad_weight is correct!
